In [1]:
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import time
import pandas as pd

In [2]:
months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

In [3]:
years = [year for year in range(2006, 2021)]

In [2]:
strainer = SoupStrainer('a', class_ = 'SiteMapArticleList-link')

In [5]:
article_links = []
length = []
t0 = time()
for year in years:
    for month in months:
        print(f'{month} - {year}')
        t1 = time()
        for day in range(1,32):
            link = f'https://www.cnbc.com/site-map/{year}/{month}/{day}/'
            page = str(requests.get(link).content)
            page = page[str(page).find('body'):]
            soup = BeautifulSoup(page, parse_only = strainer)
            links = soup.find_all('a', class_='SiteMapArticleList-link')
            if links != []:
                for a_link in links:
                    article_links.append(a_link['href'])
                    length.append(link)
        print(time()-t1)
print(time()-t0)

January - 2006
19.736795902252197
February - 2006
15.59265685081482
March - 2006
15.232767105102539
April - 2006
14.59723424911499
May - 2006
13.847558975219727
June - 2006
14.088925838470459
July - 2006
14.160164833068848
August - 2006
14.10075831413269
September - 2006
14.962288856506348
October - 2006
20.142704248428345
November - 2006
14.173299312591553
December - 2006
14.994412183761597
January - 2007
16.32174277305603
February - 2007
17.09568452835083
March - 2007
16.70344305038452
April - 2007
17.841018199920654
May - 2007
20.132397174835205
June - 2007
17.873043298721313
July - 2007
14.924350500106812
August - 2007
14.971537828445435
September - 2007
14.558979272842407
October - 2007
13.781375408172607
November - 2007
13.241758108139038
December - 2007
14.031699895858765
January - 2008
14.995840549468994
February - 2008
13.943265438079834
March - 2008
14.127214193344116
April - 2008
21.185296058654785
May - 2008
14.62737774848938
June - 2008
15.509485244750977
July - 2008
22.92

In [6]:
#Writing all the links into a csv-file to avoid rescraping every time.
with open('article_links.csv', 'w') as a_file:
    for link in article_links:
        link = ''.join(link)
        a_file.write(link + '\n')

In [5]:
def scrape_content(link):
    #Load page
    page = requests.get(link).content
    page = page[str(page).find('body'):]
    soup_link = BeautifulSoup(page)
    
    #Get title
    #title = soup_link.find('h1').get_text()
    
    #Get date for publishing
    #date = soup_link.find('meta', itemprop="datePublished")['content'][:10]
    
    #Get article content
    #article = soup_link.find('div', itemprop='text').get_text()
    
    #return [title, date, article]
    return soup_link

In [3]:
article_links = pd.read_csv('article_links.csv', sep="<", header=None)
article_links.columns = ['links']
article_links = article_links['links'].to_list()
links = []
for link in article_links:
    if ".com/advertorial" not in link and ".com/select" not in link:
        links.append(link[0:len(link)-1])
article_links = links

In [ ]:
#t0 = time()
#for link in article_links[0:10]:
#    scrape_content(link)
#    print(time()-t0)

In [4]:
strainer = SoupStrainer('div', class_='Article')

In [ ]:
df = []
index = 0
t0 = time()
for link in article_links[175000:200000]:
    t1 = time()
    request = requests.Session()
    page = request.get(link)
    status_code = page.status_code
    page = str(page.content)
    index += 1
    if status_code == 200:
        try:
            page = page[page.find('<div id="MainContent"'):]
            soup_link = BeautifulSoup(page)
            title = soup_link.find('h1', class_='ArticleHeader-headline').get_text()
            article = soup_link.find('div', class_="ArticleBody-articleBody").get_text()
            date = f'{link[29:31]}-{link[26:28]}-{link[21:25]}'
            topic = soup_link.find('a', class_="ArticleHeader-eyebrow").get_text()
            df.append([title, topic, date, article, link])
            t9 = time()
            print(f'({index}/25000) : {link}')
        except:
            pass
    else:
        print(f'({index}/25000) : Skipped')
print(time()-t0)

(1/25000) : https://www.cnbc.com/2015/03/08/troubled-china-developer-kaisa-unveils-offshore-debt-restructuring.html
(2/25000) : https://www.cnbc.com/2015/03/06/how-long-are-those-oscar-coattails-for-box-office-indie-films.html
(3/25000) : https://www.cnbc.com/2015/03/08/lax-becomes-a-battleground-between-two-retailers-over-bottled-water.html
(4/25000) : https://www.cnbc.com/2015/03/08/foreign-banks-tighten-lending-rules-for-china-state-backed-firms.html
(5/25000) : https://www.cnbc.com/2015/03/08/early-greek-election-referendum-possible-if-eu-rejects-debt-plan-varoufakis.html
(6/25000) : https://www.cnbc.com/2015/03/08/beacon-battery-on-malaysia-air-mh370-data-recorder-expired-interim-report.html
(7/25000) : https://www.cnbc.com/2015/03/08/dollar-basks-in-payrolls-afterglow-greece-back-in-hot-seat.html
(8/25000) : https://www.cnbc.com/2015/03/08/japan-4q-gdp-growth-revised-lower.html
(9/25000) : https://www.cnbc.com/2015/03/07/farfetch-the-unicorn-that-may-be-worth-1-billion.html
(10/2

(90/25000) : https://www.cnbc.com/2015/03/09/boomers-mimic-warren-buffett-when-it-comes-to-inheritances.html
(92/25000) : https://www.cnbc.com/2015/03/09/why-uk-has-to-make-business-borrowing-easier.html
(93/25000) : https://www.cnbc.com/2015/03/09/wage-growth-still-lagging-behind-clinton-reagan-years.html
(94/25000) : https://www.cnbc.com/2015/03/09/indra-nooyi-ceos-are-not-big-shots.html
(95/25000) : https://www.cnbc.com/2015/03/09/howard-schultz-key-ingredient-to-starbucks-success.html
(96/25000) : https://www.cnbc.com/2015/03/08/gold-rises-as-dollar-pauses-but-further-weakness-seen.html
(97/25000) : https://www.cnbc.com/2015/03/09/case-for-a-correction-why-even-bulls-are-nervous.html
(98/25000) : Skipped
(99/25000) : https://www.cnbc.com/2015/03/09/i-wont-be-surprised-by-a-stock-correction-siegel.html
(100/25000) : https://www.cnbc.com/2015/03/09/asian-stocks-set-for-rebound-china-inflation-eyed.html
(101/25000) : https://www.cnbc.com/2015/03/09/apple-watch-has-18-hour-battery-star

(183/25000) : https://www.cnbc.com/2015/03/10/currency-turmoil-as-us-rate-hike-jitters-bite.html
(184/25000) : https://www.cnbc.com/2015/03/10/vodafone-launches-new-policy-for-working-moms.html
(185/25000) : https://www.cnbc.com/2015/03/10/are-hot-start-ups-available-to-retail-investors.html
(186/25000) : https://www.cnbc.com/2015/03/10/french-sports-stars-among-10-dead-in-argentina-helicopter-crash.html
(187/25000) : https://www.cnbc.com/2015/03/09/qe-effect-sinks-euro-below-108-greek-worries-return.html
(188/25000) : https://www.cnbc.com/2015/03/10/marcus-lemonis.html
(189/25000) : https://www.cnbc.com/2015/03/10/cramer-remix-the-fed-still-knows-nothing.html
(191/25000) : https://www.cnbc.com/2015/03/10/marlo-thomas.html
(192/25000) : https://www.cnbc.com/2015/03/10/activists-lead-hedge-fund-bounce-back-in-february.html
(193/25000) : https://www.cnbc.com/2015/03/10/bitcoin-company-raises-166m-including-qualcomm-investment-report.html
(194/25000) : https://www.cnbc.com/2015/03/10/ford

(274/25000) : Skipped
(275/25000) : https://www.cnbc.com/2015/03/11/us-treasury-prices-fall-ahead-of-auction.html
(276/25000) : https://www.cnbc.com/2015/03/11/amazon-just-launched-a-new-marketplace.html
(277/25000) : https://www.cnbc.com/2015/03/11/bank-shares-to-jump-after-stress-tests-history.html
(278/25000) : https://www.cnbc.com/2015/03/11/under-armour-taking-technology-by-storm.html
(279/25000) : https://www.cnbc.com/2015/03/11/lightning-round-heres-how-to-look-at-yahoo.html
(280/25000) : https://www.cnbc.com/2015/03/11/chinas-retail-sales-industrial-output-miss-forecasts-in-jan-feb.html
(281/25000) : https://www.cnbc.com/2015/03/11/fed-fear-factor-could-keep-markets-on-edge.html
(282/25000) : https://www.cnbc.com/2015/03/11/these-oil-driller-stocks-could-be-winners-analyst.html
(283/25000) : https://www.cnbc.com/2015/03/11/euro-could-fall-to-85-cents-against-us-dollar-pro.html
(284/25000) : https://www.cnbc.com/2015/03/11/draghi-ecb-qe-risks-are-contained.html
(285/25000) : htt

(365/25000) : https://www.cnbc.com/2015/03/11/thomas-herzfeld-and-investing-in-cuba-a-new-fund-wades-into-the-fray.html
(366/25000) : https://www.cnbc.com/2015/03/11/uk-seen-passing-plain-cigarette-packaging-laws.html
(367/25000) : https://www.cnbc.com/2015/03/11/d-about-negative-interest-rates-cohn.html
(368/25000) : https://www.cnbc.com/2015/03/11/dollar-index-could-climb-another-25-in-2015-expert.html
(369/25000) : https://www.cnbc.com/2015/03/11/robert-chapman-long-on-lumber-liquidators.html
(370/25000) : https://www.cnbc.com/2015/03/11/power-play-two-sides-to-the-stock-trade.html
(371/25000) : https://www.cnbc.com/2015/03/11/game-of-thrones-season-5-more-than-170-coutnries-cant-escape-it-now.html
(372/25000) : https://www.cnbc.com/2015/03/11/tensions-high-as-greece-makes-ww2-payback-threats.html
(373/25000) : https://www.cnbc.com/2015/03/11/gage-applications-drop-amid-highest-rates-of-2015.html
(376/25000) : https://www.cnbc.com/2015/03/11/stop-worrying-about-stocks-its-time-to-li

...

(547/25000) : https://www.cnbc.com/2015/03/13/alleged-prostitute-accused-in-3m-extortion.html
(548/25000) : https://www.cnbc.com/2015/03/12/is-singapore-reits-stellar-run-about-to-hit-a-wall.html
(549/25000) : https://www.cnbc.com/2015/03/13/jeopardy-champ-ends-up-in-final-round-alone-after-contestants-wash-out.html
(550/25000) : https://www.cnbc.com/2015/03/13/dow-20000-by-year-end-pro-predicts.html
(551/25000) : https://www.cnbc.com/2015/03/13/they-did-what-tales-of-dumb-moves-by-smart-investors.html
(552/25000) : https://www.cnbc.com/2015/03/13/watch-this-signal-for-more-dollar-strength.html
(553/25000) : https://www.cnbc.com/2015/03/12/dollar-hits-speed-bumps-euro-gains-slight-reprieve.html
(554/25000) : https://www.cnbc.com/2015/03/13/us-doe-proposes-buying-up-to-5m-barrels-of-oil-for-strategic-reserve.html
(555/25000) : https://www.cnbc.com/2015/03/13/power-play-market-drivers-to-watch-in-the-weeks-ahead.html
(556/25000) : https://www.cnbc.com/2015/03/13/trader-bets-on-nike-earni

(635/25000) : https://www.cnbc.com/2015/03/15/siemens-ceo-greece-wont-find-solutions-in-past.html
(636/25000) : https://www.cnbc.com/2015/03/15/russia-seeing-surge-of-investment-from-china.html
(637/25000) : https://www.cnbc.com/2015/03/15/asia-hinges-on-central-bank-actions-this-week.html
(638/25000) : https://www.cnbc.com/2015/03/15/mericans-are-transforming-the-face-of-us-wealth.html
(639/25000) : https://www.cnbc.com/2015/03/14/grateful-deads-ticket-spike-shows-how-tickets-are-like-asset-prices.html
(640/25000) : https://www.cnbc.com/2015/03/15/has-the-feds-patience-been-exhausted.html
(641/25000) : https://www.cnbc.com/2015/03/15/lenty-of-room-to-maneuver-policy-premier-li.html
(642/25000) : https://www.cnbc.com/2015/03/15/what-slowdown-chinese-firms-most-upbeat-in-a-year.html
(643/25000) : https://www.cnbc.com/2015/03/15/can-ge-teach-texans-something-about-bbq.html
(644/25000) : https://www.cnbc.com/2015/03/13/pay-top-dollar-live-in-a-dorm-the-growth-of-micro-apartments.html
(645

(721/25000) : https://www.cnbc.com/2015/03/16/your-first-trade-for-tuesday-march-17.html
(722/25000) : https://www.cnbc.com/2015/03/16/99-million-car-auction-draws-mixed-results.html
(723/25000) : https://www.cnbc.com/2015/03/16/life-time-fitness-to-be-taken-private-for-over-4-bln.html
(724/25000) : https://www.cnbc.com/2015/03/16/another-week-another-euro-drubbing.html
(725/25000) : https://www.cnbc.com/2015/03/15/uk-millennials-and-mays-election-in-charts.html
(726/25000) : https://www.cnbc.com/2015/03/16/why-us-stocks-may-start-beating-europes-market.html
(727/25000) : https://www.cnbc.com/2015/03/16/uber-cfo-stepping-down-no-replacement-announced.html
(728/25000) : https://www.cnbc.com/2015/03/16/russia-isnt-the-only-bric-causing-worry--theres-also-brazil.html
(729/25000) : https://www.cnbc.com/2015/03/16/power-play-is-europe-a-risky-or-wise-bet.html
(730/25000) : https://www.cnbc.com/2015/03/16/why-the-fed-meeting-could-create-fireworks.html
(731/25000) : https://www.cnbc.com/2015

(813/25000) : https://www.cnbc.com/2015/03/17/traders-are-betting-on-this-dow-component.html
(814/25000) : https://www.cnbc.com/2015/03/17/why-foreign-investors-are-hot-on-nikkei-futures.html
(815/25000) : https://www.cnbc.com/2015/03/17/why-currency-hedged-etfs-are-hot.html
(816/25000) : https://www.cnbc.com/2015/03/17/why-apples-newest-key-market-is-cord-cutters.html
(817/25000) : https://www.cnbc.com/2015/03/17/musk-someday-driving-a-car-will-be-illegal.html
(818/25000) : https://www.cnbc.com/2015/03/17/german-economic-sentiment-highest-in-over-a-year.html
(819/25000) : https://www.cnbc.com/2015/03/17/sainsburys-sales-plunge-as-supermarket-wars-bite.html
(820/25000) : https://www.cnbc.com/2015/03/17/cial-charged-in-petrobras-bribery-case.html
(821/25000) : https://www.cnbc.com/2015/03/17/inside-donald-trumps-7-million-private-chopper.html
(822/25000) : https://www.cnbc.com/2015/03/17/after-hours-buzz-oracle-adobe-quicksilver-more.html
(823/25000) : https://www.cnbc.com/2015/03/17/pr

(902/25000) : https://www.cnbc.com/2015/03/17/how-to-get-lucky-with-your-savings.html
(903/25000) : https://www.cnbc.com/2015/03/18/us-stocks-open-lower-ahead-of-fed-statement.html
(904/25000) : https://www.cnbc.com/2015/03/17/dollar-bulls-bank-on-fed-losing-its-patience.html
(905/25000) : https://www.cnbc.com/2015/03/18/fed-removes-patient.html


In [5]:
df = pd.DataFrame(df)
df

,0,1,2,3,4
0,Taking Mom Jeans to the Next Level,Funny Business with Jane Wells,08-02-2010,Women and jeans are not a match made in heaven...,https://www.cnbc.com/2010/02/08/taking-mom-jea...
1,Same-Sex Couples Face Different Pitfalls,Love and Money 2010,08-02-2010,If you\xe2\x80\x99re a same-sex couple looking...,https://www.cnbc.com/2010/02/08/samesex-couple...
2,Watch Warren Buffett's 'Conversation' With Han...,Buffett Watch,08-02-2010,CNBC.com and CNBC-TV plan live video coverage ...,https://www.cnbc.com/2010/02/08/watch-warren-b...
3,Halftime Report: How Deep Will Correction Get?,Halftime Report,08-02-2010,Investors spent Monday struggling to make head...,https://www.cnbc.com/2010/02/08/halftime-repor...
4,When 'Dream House' Becomes Horror Home,U.S. News,09-02-2010,home with fenceIt's like a fairy tale: your dr...,https://www.cnbc.com/2010/02/09/when-dream-hou...
...,...,...,...,...,...
10149,Busch: Helping or Hurting Small Business?,The Guest Blog,16-09-2010,"With a 61-37 vote to end debate, the Senate is...",https://www.cnbc.com/2010/09/16/busch-helping-...
10150,Schork Oil Outlook: Dollar-Yen Trade Means Arm...,The Guest Blog,16-09-2010,While the dollar-euro futures have been tradin...,https://www.cnbc.com/2010/09/16/schork-oil-out...
10151,Is GameStop Under Pressure This Holiday Season?,Tech Check,16-09-2010,When a 2009 holiday price war erupted between ...,https://www.cnbc.com/2010/09/16/is-gamestop-un...
10152,A Device Sold Before Its Time,Tech Talk with David Pogue,16-09-2010,What if you were offered magical powers \xe2\x...,https://www.cnbc.com/2010/09/16/a-device-sold-...


In [6]:
df.to_csv('articles_175000_200000.csv')

In [7]:
df_test = pd.read_csv('articles_50000_61000.csv')

In [8]:
df_test.head(5)

,Unnamed: 0,0,1,2,3,4
0,0,Taking Mom Jeans to the Next Level,Funny Business with Jane Wells,08-02-2010,Women and jeans are not a match made in heaven...,https://www.cnbc.com/2010/02/08/taking-mom-jea...
1,1,Same-Sex Couples Face Different Pitfalls,Love and Money 2010,08-02-2010,If you\xe2\x80\x99re a same-sex couple looking...,https://www.cnbc.com/2010/02/08/samesex-couple...
2,2,Watch Warren Buffett's 'Conversation' With Han...,Buffett Watch,08-02-2010,CNBC.com and CNBC-TV plan live video coverage ...,https://www.cnbc.com/2010/02/08/watch-warren-b...
3,3,Halftime Report: How Deep Will Correction Get?,Halftime Report,08-02-2010,Investors spent Monday struggling to make head...,https://www.cnbc.com/2010/02/08/halftime-repor...
4,4,When 'Dream House' Becomes Horror Home,U.S. News,09-02-2010,home with fenceIt's like a fairy tale: your dr...,https://www.cnbc.com/2010/02/09/when-dream-hou...
